In [1]:
# 기본
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
from matplotlib import font_manager, rc
import platform
from tqdm import tqdm
import sklearn
from sklearn import linear_model
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from patsy import dmatrices

# crawling
import requests
import lxml.html
import sqlite3
from pandas.io import sql
from bs4 import BeautifulSoup

# 한글 폰트 설정
if platform.system() == 'Windows':
# 윈도우인 경우
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)

In [2]:
wind = pd.read_csv('0. data\\thermal inversion\\기상청_풍향_2002_2021.csv', encoding = 'euc-kr')
wind

,지점,지점명,일시,풍속(m/s),풍속 QC플래그,풍향(16방위),풍향 QC플래그
0,108,서울,2002-01-01 0:00,2.2,0.0,290.0,0.0
1,108,서울,2002-01-01 1:00,3.2,0.0,290.0,0.0
2,108,서울,2002-01-01 2:00,2.9,0.0,290.0,0.0
3,108,서울,2002-01-01 3:00,2.9,0.0,290.0,0.0
4,108,서울,2002-01-01 4:00,3.5,0.0,290.0,0.0
...,...,...,...,...,...,...,...
175309,108,서울,2021-12-31 19:00,2.2,NaN,250.0,NaN
175310,108,서울,2021-12-31 20:00,4.1,NaN,320.0,NaN
175311,108,서울,2021-12-31 21:00,4.8,NaN,320.0,NaN
175312,108,서울,2021-12-31 22:00,3.0,NaN,320.0,NaN


In [17]:
wind['일시'].str[11:]

0          0:00
1          1:00
2          2:00
3          3:00
4          4:00
          ...  
175309    19:00
175310    20:00
175311    21:00
175312    22:00
175313    23:00
Name: 일시, Length: 175314, dtype: object

In [23]:
wind_1417 = wind.loc[wind['일시'].str[0:4].isin(['2014', '2015', '2016', '2017']) == True].reset_index(drop = True)
wind_1417 = wind_1417.loc[wind_1417['일시'].str[11:].isin(['9:00']) == True].reset_index(drop = True)
wind_1417['year_month'] = wind_1417['일시'].str[0:7]
wind_1417['year'] = wind_1417['일시'].str[0:4]
wind_1417

,지점,지점명,일시,풍속(m/s),풍속 QC플래그,풍향(16방위),풍향 QC플래그,year_month,year
0,108,서울,2014-01-01 9:00,2.3,0.0,230.0,0.0,2014-01,2014
1,108,서울,2014-01-02 9:00,0.8,0.0,20.0,0.0,2014-01,2014
2,108,서울,2014-01-03 9:00,1.7,0.0,20.0,0.0,2014-01,2014
3,108,서울,2014-01-04 9:00,1.7,0.0,270.0,0.0,2014-01,2014
4,108,서울,2014-01-05 9:00,0.2,0.0,0.0,0.0,2014-01,2014
...,...,...,...,...,...,...,...,...,...
1456,108,서울,2017-12-27 9:00,2.0,NaN,320.0,NaN,2017-12,2017
1457,108,서울,2017-12-28 9:00,0.7,0.0,50.0,0.0,2017-12,2017
1458,108,서울,2017-12-29 9:00,0.2,0.0,0.0,0.0,2017-12,2017
1459,108,서울,2017-12-30 9:00,1.3,NaN,90.0,NaN,2017-12,2017


In [22]:
year_month = wind_1417['year_month'].unique().tolist()
for ym in year_month: 
    print(ym)
    tmp = wind_1417.loc[wind_1417['year_month'] == ym]
    for col in tmp.columns: 
        print(col, tmp[col].isna().sum())
    print('---------------------------------------------')

2014-01
지점 0
지점명 0
일시 0
풍속(m/s) 0
풍속 QC플래그 0
풍향(16방위) 0
풍향 QC플래그 0
year_month 0
---------------------------------------------
2014-02
지점 0
지점명 0
일시 0
풍속(m/s) 0
풍속 QC플래그 0
풍향(16방위) 1
풍향 QC플래그 0
year_month 0
---------------------------------------------
2014-03
지점 0
지점명 0
일시 0
풍속(m/s) 0
풍속 QC플래그 0
풍향(16방위) 0
풍향 QC플래그 0
year_month 0
---------------------------------------------
2014-04
지점 0
지점명 0
일시 0
풍속(m/s) 0
풍속 QC플래그 0
풍향(16방위) 0
풍향 QC플래그 0
year_month 0
---------------------------------------------
2014-05
지점 0
지점명 0
일시 0
풍속(m/s) 0
풍속 QC플래그 0
풍향(16방위) 0
풍향 QC플래그 0
year_month 0
---------------------------------------------
2014-06
지점 0
지점명 0
일시 0
풍속(m/s) 0
풍속 QC플래그 0
풍향(16방위) 0
풍향 QC플래그 0
year_month 0
---------------------------------------------
2014-07
지점 0
지점명 0
일시 0
풍속(m/s) 0
풍속 QC플래그 0
풍향(16방위) 0
풍향 QC플래그 0
year_month 0
---------------------------------------------
2014-08
지점 0
지점명 0
일시 0
풍속(m/s) 0
풍속 QC플래그 0
풍향(16방위) 0
풍향 QC플래그 0
year_month 0
--------------------------------------

In [24]:
year = wind_1417['year'].unique().tolist()
for y in year: 
    print(y)
    tmp = wind_1417.loc[wind_1417['year'] == y]
    for col in tmp.columns: 
        print(col, tmp[col].isna().sum())
    print('---------------------------------------------')

2014
지점 0
지점명 0
일시 0
풍속(m/s) 0
풍속 QC플래그 0
풍향(16방위) 1
풍향 QC플래그 0
year_month 0
year 0
---------------------------------------------
2015
지점 0
지점명 0
일시 0
풍속(m/s) 0
풍속 QC플래그 1
풍향(16방위) 0
풍향 QC플래그 1
year_month 0
year 0
---------------------------------------------
2016
지점 0
지점명 0
일시 0
풍속(m/s) 0
풍속 QC플래그 2
풍향(16방위) 0
풍향 QC플래그 2
year_month 0
year 0
---------------------------------------------
2017
지점 0
지점명 0
일시 0
풍속(m/s) 1
풍속 QC플래그 35
풍향(16방위) 1
풍향 QC플래그 35
year_month 0
year 0
---------------------------------------------
